<a href="https://colab.research.google.com/github/harshita23sharma/computer_vision/blob/main/custom_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import sys
sys.path.append('/CIFAR_10/')
# from CIFAR_10 import utils
# from CIFAR_10 import main
# from CIFAR_10.GradCAM.visualize import VisualizeCam

!pip install torchsummary
from torchsummary import summary

import torch

In [141]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
# from CIFAR_10.models.reimport torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride):
        print("in_planes: ",in_planes)
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)

        self.maxpool1 = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)


        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                self.conv2,
                self.bn2,
                nn.ReLU()
                
            )

    def forward(self, x):
        out1 = F.relu(self.bn1(self.maxpool1(self.conv1(x))))
        out = F.relu(self.bn2(self.conv2(out1)))
        out = self.shortcut(out)
        # F.relu(self.bn2(self.conv2(out)))
        out += out1
        print("OUT:",out.shape)
        out = F.relu(out)
        return out

class BasicBlock2(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        print("in_planes: ",in_planes)
        super(BasicBlock2, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes, 1)
        self.max_pool = nn.MaxPool2d(2,2)

    def forward(self, x):
        out = F.relu(self.bn1(self.max_pool(self.conv1(x))))
        return out



class ResNetCustom(nn.Module):
    def __init__(self, block, block2, num_blocks, num_classes=10):
        super(ResNetCustom, self).__init__()

        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        # self._prep_layer = F.relu(self.bn1(self.conv1(x)))

        self.layer1 = self._make_layer(block, 128, num_blocks[0], stride=1)

        self.layer2 = self._make_layer2(block2, 256, num_blocks[1], stride=1)

        self.conv2 = nn.Conv2d(128, 256, kernel_size=3, stride=3, bias=False)
        self.max_pool = nn.MaxPool2d(4,2)
        self.bn2 = nn.BatchNorm2d(256)

        self.layer3 = self._make_layer(block, 512, num_blocks[2], stride=1)

        self.max_pool2 = nn.MaxPool2d(4)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)
    
    def _make_layer2(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        r1 = self.layer1(x)
        print("r1 : ",r1.shape)
        out = self.layer2(r1)
        print(out.shape) #([2, 128, 32, 32])
        r2 = self.layer3(out)
        out = self.max_pool2(r2)
        out = out.view(out.size(0), -1)
        print("out shape",out.shape)
        out = self.linear(out)
        return out


def CustomResNet():
    return ResNetCustom(BasicBlock,BasicBlock2, [1, 1, 1],)


def test():
    net = CustomResNet()
    print(summary(net, input_size=(3, 32, 32)))
    y = net(torch.randn(1, 3, 32, 32))
    print(y.size())



test()

in_planes:  64
in_planes:  128
in_planes:  256
OUT: torch.Size([2, 128, 16, 16])
r1 :  torch.Size([2, 128, 16, 16])
torch.Size([2, 256, 8, 8])
OUT: torch.Size([2, 512, 4, 4])
out shape torch.Size([2, 512])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3          [-1, 128, 32, 32]          73,728
         MaxPool2d-4          [-1, 128, 16, 16]               0
       BatchNorm2d-5          [-1, 128, 16, 16]             256
            Conv2d-6          [-1, 128, 16, 16]         147,456
            Conv2d-7          [-1, 128, 16, 16]         147,456
       BatchNorm2d-8          [-1, 128, 16, 16]             256
       BatchNorm2d-9          [-1, 128, 16, 16]             256
           Conv2d-10          [-1, 128, 16, 16]         147,456
           Conv2d-11     

In [103]:
import os
import argparse


# Training
def train(epoch, net, criterion, optimizer, device, trainloader, train_losses, train_acc):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print('Train Loss: %.3f | Train Acc: %.3f%% (%d/%d)' % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    train_losses.append(train_loss/(batch_idx+1))
    train_acc.append(100.*correct/total)
    return train_losses, train_acc


def test(epoch, net, criterion, device, testloader, best_acc, test_losses, test_acc):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    print('Test Loss: %.3f | Test Acc: %.3f%% (%d/%d)' % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
    
    test_losses.append(test_loss/(batch_idx+1))
    test_acc.append(100.*correct/total)
    
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        best_acc = acc
    return best_acc, test_losses, test_acc
    


def dataloaders(trainset, testset):
    trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=4)

    testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=4)
    
    return trainloader, testloader


def start_training(no_of_epoch, net, criterion, optimizer, device, trainloader, testloader, best_acc, scheduler):
    train_loss = []
    train_acc = []
    test_loss = []
    test_acc = []
    
    for epoch in range(no_of_epoch):
        train_loss, train_acc = train(epoch+1, net, criterion, optimizer, device, trainloader, train_loss, train_acc)
        best_acc, test_loss, test_acc = test(epoch+1, net, criterion, device, testloader, best_acc, test_loss, test_acc)
        scheduler.step(test_loss[-1])
    print("Best Acc is : ", best_acc)
    return train_loss, train_acc, test_loss, test_acc

        
def define_model_utilities(loss="cross_entropy", optimizer_func="SGD", lr=0.1):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    best_acc = 0  # best test accuracy
    classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

    net = CustomResNet()
    net = net.to(device)
     
    if loss=="cross_entropy":
        criterion = nn.CrossEntropyLoss()
    
    if optimizer_func=="SGD":
        optimizer = optim.SGD(net.parameters(), lr=lr,
                      momentum=0.9, weight_decay=5e-4)
        
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3)
    
    return device, best_acc, classes, net, criterion, optimizer, scheduler


In [104]:
device, best_acc, classes, net, criterion, optimizer, scheduler = define_model_utilities(loss="cross_entropy", optimizer_func="SGD", lr=0.1)

in_planes:  64
in_planes:  128
in_planes:  128
in_planes:  256
in_planes:  512


In [105]:
summary(net, input_size=(3, 32, 32))

torch.Size([2, 256, 6, 6])
out shape torch.Size([2, 512])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3          [-1, 128, 16, 16]          73,728
       BatchNorm2d-4          [-1, 128, 16, 16]             256
            Conv2d-5          [-1, 128, 16, 16]         147,456
       BatchNorm2d-6          [-1, 128, 16, 16]             256
            Conv2d-7          [-1, 128, 16, 16]           8,192
       BatchNorm2d-8          [-1, 128, 16, 16]             256
        BasicBlock-9          [-1, 128, 16, 16]               0
           Conv2d-10          [-1, 128, 16, 16]         147,456
      BatchNorm2d-11          [-1, 128, 16, 16]             256
           Conv2d-12          [-1, 128, 16, 16]         147,456
      BatchNorm2d-13          [-1, 128, 16, 1